In [1]:
#Importation des packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

## FICHIER UNIQUE DE DONNEES

In [2]:
#Fichier INCIDENT 2009_2017
df1 = pd.read_csv("0.1_initial_Incidents_2009 - 2017.csv",low_memory=False)
pd.set_option('display.max_columns', None)
#df1.head()

In [3]:
#Fichier INCIDENT 2018-2024:
df2 = pd.read_csv("0.2_initial_Incidents_2018  - 2024.csv",sep=";",low_memory=False)

In [4]:
# Fichier final Incident entre 2009 et 2024
df = pd.concat([df1, df2], axis = 0)

In [5]:
#df.shape

In [6]:
#Fichier MOBILISATION 2009-2014
data1 = pd.read_csv("0.3_initial_Mobilisation_2009 - 2014.csv",sep=";")
#data1.shape

In [7]:
#Fichier MOBILISATION 2015-2020 
data2 = pd.read_csv("0.4_initial_ Mobilisation_2015 - 2020.csv", sep=';', low_memory=False)

In [8]:
#Fichier MOBILISATION 2021-2024
data3 = pd.read_csv("0.4_initial_Mobilisation_2021 - 2024.csv", sep=";",low_memory=False)

In [9]:
#Concaténer les fichiers mobilisation 
data = pd.concat([data1, data2, data3], axis = 0)

In [10]:
#Format de IncidentNumber avant la fusion:
data["IncidentNumber"] = data["IncidentNumber"].astype(str)
df["IncidentNumber"] = df["IncidentNumber"].astype(str)
df["IncidentNumber"] = df["IncidentNumber"].astype(str).str.replace(r'\.00$', '', regex=True)

In [11]:
#Fusionner Incidents et Mobilisation
LFB = pd.merge(data, df, on=["IncidentNumber", "CalYear", "HourOfCall"], how="inner")

In [12]:
LFB.shape

(2488987, 60)

In [13]:
#LFB.head()

#### DONNEES TEMPORELLES

In [14]:
#Création d'une colonne temporelle au format d/m/Y 
LFB["DATE"] = LFB["DateAndTimeMobilised"].apply(lambda x : x[:10])

In [15]:
#Mettre les variables temporelles au type datetime
LFB["DATE"] =  pd.to_datetime(LFB["DATE"],format = '%d/%m/%Y')
LFB["DateAndTimeMobile"] = pd.to_datetime(LFB["DateAndTimeMobile"],format = '%d/%m/%Y %H:%M', errors ='coerce')
LFB["DateAndTimeMobilised"] = pd.to_datetime(LFB["DateAndTimeMobilised"],format = '%d/%m/%Y %H:%M', errors ='coerce')
LFB["DateAndTimeArrived"] = pd.to_datetime(LFB["DateAndTimeArrived"],format = '%d/%m/%Y %H:%M', errors ='coerce')
LFB["DateAndTimeLeft"] = pd.to_datetime(LFB["DateAndTimeLeft"],format = '%d/%m/%Y %H:%M', errors ='coerce')
LFB["DateAndTimeReturned"] = pd.to_datetime(LFB["DateAndTimeReturned"],format = '%d/%m/%Y %H:%M', errors ='coerce')

In [16]:
#Création de nouvelles variables temporelles
LFB.loc[:,"Month"] = LFB["DateAndTimeMobilised"].dt.month
LFB.loc[:,"WeekDay"] = LFB["DateAndTimeMobilised"].dt.day_name()
LFB.loc[:,"Day"] = LFB["DateAndTimeMobilised"].dt.day

#### SUPPRESSION DE LIGNES NON PERTINENTES

In [17]:
#Suppression des doublons
LFB.duplicated().sum()
LFB=LFB.drop_duplicates()

In [18]:
#Conserver le premier camion mobilisé
filtered_LFB = LFB[LFB["PumpOrder"]==1]
LFB = filtered_LFB.drop_duplicates(subset="IncidentNumber", keep="first").reset_index(drop=True)

In [19]:
#Suppression des lignes correspondantes à la période du confinement COVID-19 
LFB = LFB.loc[(LFB["DATE"]<"23-03-2020")|(LFB["DATE"]>"21-02-2021")]

In [20]:
#Suppression des lignes NaN de la variable-cible
LFB = LFB.dropna(subset = "FirstPumpArriving_AttendanceTime")

#### ENRICHISSEMENT DE DONNEES

In [21]:
#Fichiers Météo
meteo09 = pd.read_csv("export-londres2009.csv", sep=",", skiprows=3)
meteo10 = pd.read_csv("export-londres2010.csv", sep=",", skiprows=3)
meteo11 = pd.read_csv("export-londres2011.csv", sep=",", skiprows=3)
meteo12 = pd.read_csv("export-londres2012.csv", sep=",", skiprows=3)
meteo13 = pd.read_csv("export-londres2013.csv", sep=",", skiprows=3)
meteo14 = pd.read_csv("export-londres2014.csv", sep=",", skiprows=3)
meteo15 = pd.read_csv("export-londres2015.csv", sep=",", skiprows=3)
meteo16 = pd.read_csv("export-londres2016.csv", sep=",", skiprows=3)
meteo17 = pd.read_csv("export-londres2017.csv", sep=",", skiprows=3)
meteo18 = pd.read_csv("export-londres2018.csv", sep=",", skiprows=3)
meteo19 = pd.read_csv("export-londres2019.csv", sep=",", skiprows=3)
meteo20 = pd.read_csv("export-londres2020.csv", sep=",", skiprows=3)
meteo21 = pd.read_csv("export-londres2021.csv", sep=",", skiprows=3)
meteo22 = pd.read_csv("export-londres2022.csv", sep=",", skiprows=3)
meteo23 = pd.read_csv("export-londres2023.csv", sep=",", skiprows=3)
meteo24 = pd.read_csv("export-londres2024.csv", sep=",", skiprows=3)

In [22]:
meteo = pd.concat([meteo09,meteo10,meteo11,meteo12,meteo13,meteo14,meteo15,meteo16,meteo17,meteo18,meteo19,meteo20,meteo21,meteo22,meteo23,meteo24], axis=0)

In [23]:
#Conserver les variables pertinentes pour le projet
meteo = meteo[["DATE", "OPINION", "VISIBILITY_AVG_KM"]]

In [24]:
#Conversion DATE en datetime avant fusion
meteo["DATE"] =  pd.to_datetime(meteo["DATE"],format = '%Y-%m-%d')

In [25]:
#Suppression des dates postérieures au 29 nov 2024 qui est la dernière date de notre jeu de données
meteo = meteo.loc[(meteo["DATE"]>= "2009-01-01")&(meteo["DATE"]<="2024-11-29")]

In [26]:
#Suppression de la période de confinement : 
meteo = meteo.loc[(meteo["DATE"]<"2020-03-23")|(meteo["DATE"]>"2021-02-21")]

In [27]:
#meteo.duplicated().sum()
#meteo.isna().sum()

In [28]:
#Fusion du fichier LFB et meteo
LFB_Meteo = pd.merge(LFB,meteo, on="DATE",how="inner")

In [29]:
LFB_Meteo.shape

(1560925, 66)

In [31]:
#LFB_Meteo.isna().sum()

In [33]:
LFB_Meteo.duplicated().sum()

0

In [34]:
#LFB_Meteo.dtypes

In [35]:
#Création d'une variable catégorielle de la visibilité sur la route
LFB_Meteo.loc[(LFB_Meteo ["VISIBILITY_AVG_KM"]<= 2.0, ["Visibility"])] = "très mauvaise"
LFB_Meteo.loc[(LFB_Meteo ["VISIBILITY_AVG_KM"]>2.0)&(LFB_Meteo["VISIBILITY_AVG_KM"]<=10),["Visibility"]] = "moyenne"
LFB_Meteo.loc[(LFB_Meteo ["VISIBILITY_AVG_KM"]>10), ["Visibility"]] = "très bonne"

In [36]:
#Suppression de la variable initiale de visibilité :
LFB_Meteo = LFB_Meteo.drop("VISIBILITY_AVG_KM", axis=1)

## EXPLORATION 

In [37]:
#Variables catégorielles :
LFB_Meteo["CalYear"] = LFB_Meteo["CalYear"].astype(object)
LFB_Meteo["HourOfCall"] = LFB_Meteo["HourOfCall"].astype(object)
LFB_Meteo["UPRN"] = LFB_Meteo["UPRN"].astype(object)
LFB_Meteo["USRN"] = LFB_Meteo["USRN"].astype(object)
LFB_Meteo["Easting_m"] = LFB_Meteo["Easting_m"].astype(object)
LFB_Meteo["Northing_m"] = LFB_Meteo["Northing_m"].astype(object)
LFB_Meteo["Easting_rounded"] = LFB_Meteo["Easting_rounded"].astype(object)
LFB_Meteo["Northing_rounded"] = LFB_Meteo["Northing_rounded"].astype(object)
LFB_Meteo["Latitude"] = LFB_Meteo["Latitude"].astype(object)
LFB_Meteo["Longitude"] = LFB_Meteo["Longitude"].astype(object)
LFB_Meteo["ResourceMobilisationId"] = LFB_Meteo["ResourceMobilisationId"].astype(object)
LFB_Meteo["PerformanceReporting"] = LFB_Meteo["PerformanceReporting"].astype(object)
LFB_Meteo["PumpOrder"] = LFB_Meteo["PumpOrder"].astype(object)
LFB_Meteo["DelayCodeId"] = LFB_Meteo["DelayCodeId"].astype(object)
LFB_Meteo["Month"] = LFB_Meteo["Month"].astype(object)
LFB_Meteo["Day"] = LFB_Meteo["Day"].astype(object)

#### DESCRIPTION

In [38]:
#Mettre les variables temporelles au type datetime
LFB_Meteo["DATE"] =  pd.to_datetime(LFB_Meteo["DATE"],format = '%d/%m/%Y')
LFB_Meteo["DateAndTimeMobile"] = pd.to_datetime(LFB_Meteo["DateAndTimeMobile"],format = '%d/%m/%Y %H:%M', errors ='coerce')
LFB_Meteo["DateAndTimeMobilised"] = pd.to_datetime(LFB_Meteo["DateAndTimeMobilised"],format = '%d/%m/%Y %H:%M', errors ='coerce')
LFB_Meteo["DateAndTimeArrived"] = pd.to_datetime(LFB_Meteo["DateAndTimeArrived"],format = '%d/%m/%Y %H:%M', errors ='coerce')
LFB_Meteo["DateAndTimeLeft"] = pd.to_datetime(LFB_Meteo["DateAndTimeLeft"],format = '%d/%m/%Y %H:%M', errors ='coerce')
LFB_Meteo["DateAndTimeReturned"] = pd.to_datetime(LFB_Meteo["DateAndTimeReturned"],format = '%d/%m/%Y %H:%M', errors ='coerce')

In [39]:
LFB_Meteo.dtypes
pd.set_option('display.max_rows', None)
LFB_Meteo.dtypes

IncidentNumber                                    object
CalYear                                           object
HourOfCall                                        object
ResourceMobilisationId                            object
Resource_Code                                     object
PerformanceReporting                              object
DateAndTimeMobilised                      datetime64[ns]
DateAndTimeMobile                         datetime64[ns]
DateAndTimeArrived                        datetime64[ns]
TurnoutTimeSeconds                               float64
TravelTimeSeconds                                float64
AttendanceTimeSeconds                              int64
DateAndTimeLeft                           datetime64[ns]
DateAndTimeReturned                       datetime64[ns]
DeployedFromStation_Code                          object
DeployedFromStation_Name                          object
DeployedFromLocation                              object
PumpOrder                      

In [40]:
LFB_Meteo.describe(include='number').round(2)

,TurnoutTimeSeconds,TravelTimeSeconds,AttendanceTimeSeconds,FirstPumpArriving_AttendanceTime,SecondPumpArriving_AttendanceTime,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,Notional Cost (£),NumCalls
count,1539320.00,1539234.0,1560925.00,1560925.00,601467.00,1560925.00,1560925.00,1560925.00,1560925.00,1560925.00,1559931.00
mean,77.38,244.2,319.38,319.38,396.98,1.39,1.58,1.63,76.23,404.20,1.33
std,42.94,136.0,139.43,139.43,153.97,0.69,0.85,1.44,325.63,1742.35,1.51
min,0.00,0.0,1.00,1.00,1.00,1.00,1.00,1.00,60.00,255.00,1.00
25%,53.00,156.0,231.00,231.00,296.00,1.00,1.00,1.00,60.00,260.00,1.00
50%,72.00,221.0,297.00,297.00,371.00,1.00,1.00,1.00,60.00,328.00,1.00
75%,94.00,303.0,380.00,380.00,465.00,2.00,2.00,2.00,60.00,364.00,1.00
max,1192.00,1198.0,1200.00,1200.00,1200.00,46.00,106.00,402.00,129751.00,709305.00,369.00


In [41]:
#Renommer des variables
LFB_Meteo  = LFB_Meteo.rename(columns={"Notional Cost (£)":"NationalCost","OPINION":"Meteo"})

#### OUTLIERS

In [42]:
#Suppression des valeurs >780s de la variable cible : 
LFB_Meteo = LFB_Meteo[LFB_Meteo["FirstPumpArriving_AttendanceTime"]<780]

In [43]:
#LFB_Meteo.shape

#### CORRELATION

In [44]:
#Etude de la corrélation entre le temps de réponse et la visibilité
import statsmodels.api as sm
import statsmodels.formula.api as smf
result = smf.ols("FirstPumpArriving_AttendanceTime~Visibility", data = LFB_Meteo).fit()
anova_results = sm.stats.anova_lm(result)
anova_results

,df,sum_sq,mean_sq,F,PR(>F)
Visibility,1.0,1.738774e+06,1.738774e+06,117.457486,2.284532e-27
Residual,1541276.0,2.281618e+10,1.480344e+04,NaN,NaN


In [45]:
#Test de corrélation entre le temps d'arrivée de la première pompe et la météo:
result = smf.ols("FirstPumpArriving_AttendanceTime~Meteo", data = LFB_Meteo).fit()
anova_results = sm.stats.anova_lm(result)
anova_results

,df,sum_sq,mean_sq,F,PR(>F)
Meteo,4.0,3.433966e+06,858491.588520,57.996924,4.970507e-49
Residual,1541273.0,2.281448e+10,14802.364086,NaN,NaN


#### SELECTION DES VARIABLES DE L'ETUDE

In [46]:
LFB_Fusion = LFB_Meteo[["DeployedFromStation_Name","Easting_rounded","Northing_rounded",
                    "Meteo","Visibility","StopCodeDescription","PropertyCategory",
                    "CalYear","HourOfCall","FirstPumpArriving_AttendanceTime"]]

In [47]:
LFB_Fusion.shape

(1541278, 10)